<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-10 09:54:09
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 33.71 K (0.25%)
Current PnL: -12.70 L (-8.85%)
CY Booked + Current PnL: -5.74 L (-4.0%)
-------------------
Total profit:  4.95 L
Total loss:  -17.65 L
-------------------
Total Booked + Current PnL: 21.24 L (18.23%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.06%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.20 L (51.81%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 11.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-220.24,H-SC,98.96,86707.0,-8609.0,8645.0,1.51,-9.03,9.97,0.04,157.0,-1.00,0.63,71.02,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-18.73,H-LC,90.62,110790.0,7224.0,13483.0,-1.11,6.97,12.17,20.00,32.0,0.54,0.81,22.03,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-21.55,H-LC,70.83,108056.0,3744.0,13788.0,0.21,3.59,12.76,16.81,27.0,0.27,0.79,20.21,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,43.75,214822.0,13342.0,13878.0,-0.04,6.62,6.46,13.51,77.0,0.96,1.56,20.05,XY25,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,6.25,128871.0,-13608.0,14227.0,0.40,-9.55,11.04,0.44,247.0,-0.96,0.94,52.10,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,38.54,151277.0,364.0,60087.0,-0.25,0.24,39.72,40.06,103.0,0.01,1.1,23.74,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.0,-21.05,M-SC,53.12,164356.0,-3419.0,59267.0,6.71,-2.04,36.06,33.28,206.0,-0.06,1.20,27.25,XY24,NTT,FOOTWEAR
65,RELAXO,902.64,1176.0,-26.60,H-SC,95.83,66406.0,-49132.0,84123.0,3.86,-42.52,126.68,30.28,131.0,-0.58,0.48,29.73,X40N,NTT,FOOTWEAR
52,JPPOWER,18.73,26.2,-13.04,L-SC,100.00,175239.0,33397.0,23167.0,3.44,23.54,13.22,39.88,262.0,1.44,1.28,77.59,XY24,NTT,POWER
75,SIS,477.00,477.0,2250.26,M-SC,44.79,63232.0,-17858.0,17857.0,2.75,-22.02,28.24,0.00,232.0,-1.00,0.46,28.15,OX40N,NTT,MISC
64,RAJESHEXPO,517.65,518.0,1910.10,M-SC,65.62,54858.0,-82319.0,82413.0,1.98,-60.01,150.23,0.07,235.0,-1.00,0.40,36.43,OX40N,NTT,JEWELLERY


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,LALPATHLAB,2804.19,3545.00,-30.08,H-SC,41.67,160880.0,6650.0,34090.0,-1.30,4.31,21.19,26.42,127.0,0.20,1.17,26.87,X40N,NTT,HEALTHCARE
43,ICICIGI,1839.64,2260.25,-9.98,H-LC,55.21,147660.0,11527.0,19594.0,-1.23,8.47,13.27,22.86,35.0,0.59,1.08,22.15,X40,ATH,INSURANCE
22,CIPLA,1495.00,1795.00,-23.18,H-LC,30.21,98805.0,-1360.0,21460.0,-1.14,-1.36,21.72,20.07,30.0,-0.06,0.72,6.50,X40N,BTT,PHARMA
4,APOLLOHOSP,6904.43,8285.00,-18.73,H-LC,90.62,110790.0,7224.0,13483.0,-1.11,6.97,12.17,20.00,32.0,0.54,0.81,22.03,X40N,BTT,HEALTHCARE
62,PGHH,13388.00,18062.58,-20.87,X-MC,36.46,199515.0,-1305.0,71426.0,-0.86,-0.65,35.80,34.92,55.0,-0.02,1.45,4.00,X40,ATH,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-220.24,H-SC,98.96,86707.0,-8609.0,8645.0,1.51,-9.03,9.97,0.04,157.0,-1.00,0.63,71.02,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,6.25,128871.0,-13608.0,14227.0,0.40,-9.55,11.04,0.44,247.0,-0.96,0.94,52.10,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.61,M-SC,32.29,152235.0,-18906.0,18847.0,0.75,-11.05,12.38,-0.03,191.0,-1.00,1.11,9.42,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,91.67,148936.0,-25623.0,25662.0,0.07,-14.68,17.23,0.02,133.0,-1.00,1.08,29.15,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.15,X-MC,5.21,85925.0,-16727.0,16721.0,-0.24,-16.29,19.46,-0.00,59.0,-1.00,0.63,30.62,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.02,X-LC,31.25,263991.0,-31943.0,125845.0,0.02,-10.79,47.67,31.73,1.0,-0.25,1.92,5.67,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.55,X-LC,60.94,290660.0,29059.0,116555.0,-0.61,11.11,40.10,55.67,2.0,0.25,2.12,17.89,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.47,X-LC,66.67,271219.0,872.0,33767.0,-0.07,0.32,12.45,12.81,5.0,0.03,1.97,13.43,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.76,X-LC,76.04,269905.0,40672.0,26613.0,-0.25,17.74,9.86,29.35,10.0,1.53,1.97,28.25,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-4.80,X-LC,15.62,80420.0,-29752.0,118829.0,-0.05,-27.01,147.76,80.85,11.0,-0.25,0.59,18.45,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3183.4,NaN,NaN,96.88,79585.0,45136.0,0.0,1.82,131.02,0.00,131.02,274.0,inf,0.58,21.70,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-13.04,L-SC,100.00,175239.0,33397.0,23167.0,3.44,23.54,13.22,39.88,262.0,1.44,1.28,77.59,XY24,NTT,POWER


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,59.38,232074.0,7112.0,160549.0,0.66,3.16,69.18,74.53,189.0,0.04,1.69,34.74,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,29.10,H-SC,57.29,132199.0,6902.0,52205.0,1.40,5.51,39.49,47.18,143.0,0.13,0.96,65.39,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.76,H-SC,37.50,117617.0,15613.0,29416.0,0.23,15.31,25.01,44.14,126.0,0.53,0.86,34.03,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-3.74,H-SC,93.75,180660.0,15990.0,53096.0,-0.17,9.71,29.39,41.95,124.0,0.30,1.32,17.90,X40N,ATH,AC
95,WIPRO,243.46,420.00,-3.77,M-LC,80.21,164765.0,13820.0,95630.0,-0.77,9.16,58.04,72.51,71.0,0.14,1.20,13.50,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3183.40,NaN,NaN,96.88,79585.0,45136.0,0.0,1.82,131.02,0.00,131.02,274.0,inf,0.58,21.70,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-39.44,M-SC,73.96,199729.0,28108.0,133958.0,0.80,16.38,67.07,94.43,201.0,0.21,1.45,47.41,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,80.21,164765.0,13820.0,95630.0,-0.77,9.16,58.04,72.51,71.0,0.14,1.20,13.50,XR,NTT,IT
54,KPIGREEN,497.21,731.79,29.10,H-SC,57.29,132199.0,6902.0,52205.0,1.40,5.51,39.49,47.18,143.0,0.13,0.96,65.39,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-47.52,H-SC,75.00,136655.0,13319.0,120448.0,-0.36,10.80,88.14,108.45,123.0,0.11,1.00,35.57,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.14,NaN,NaN,63.02,18865.0,-30728.0,0.0,0.13,-61.96,0.00,-61.96,270.0,-inf,0.14,26.51,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,13.54,66182.0,-47367.0,87420.0,0.35,-41.71,132.09,35.27,265.0,-0.54,0.48,61.98,XR,NTT,HOTELS
32,ENRIN,1377.95,3183.40,NaN,NaN,96.88,79585.0,45136.0,0.0,1.82,131.02,0.00,131.02,274.0,inf,0.58,21.70,AR,ATH,ELECTRICAL
92,VIPIND,488.80,489.00,-220.24,H-SC,98.96,86707.0,-8609.0,8645.0,1.51,-9.03,9.97,0.04,157.0,-1.00,0.63,71.02,OX40N,NTT,MISC
58,MASFIN,326.60,399.50,-16.89,H-SC,82.29,94650.0,-3330.0,25196.0,0.94,-3.40,26.62,22.32,145.0,-0.13,0.69,37.28,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.04,L-SC,100.00,175239.0,33397.0,23167.0,3.44,23.54,13.22,39.88,262.0,1.44,1.28,77.59,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-220.24,H-SC,98.96,86707.0,-8609.0,8645.0,1.51,-9.03,9.97,0.04,157.0,-1.00,0.63,71.02,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,8.78,X-SC,97.92,101312.0,349.0,94180.0,1.34,0.35,92.96,93.63,67.0,0.00,0.74,50.90,SR,ATH,CHEMICALS
32,ENRIN,1377.95,3183.40,NaN,NaN,96.88,79585.0,45136.0,0.0,1.82,131.02,0.00,131.02,274.0,inf,0.58,21.70,AR,ATH,ELECTRICAL
65,RELAXO,902.64,1176.00,-26.60,H-SC,95.83,66406.0,-49132.0,84123.0,3.86,-42.52,126.68,30.28,131.0,-0.58,0.48,29.73,X40N,NTT,FOOTWEAR


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.90
1,25,41.85
2,50,70.61


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.79
LC    33.67
MC    21.49
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.12
X40      12.87
XY25     12.04
X40N     11.92
XR        9.99
OX40N     8.21
AR        8.18
MH        1.68
X5K       1.41
X200      1.39
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.59
M-SC    15.98
H-LC    14.01
X-LC    12.84
H-MC     9.56
X-MC     7.54
M-LC     5.78
X-SC     4.55
M-MC     4.04
L-SC     2.67
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.20,-0.14,33.62
IT,12.01,-10.30,64.51
FINANCE,8.54,-11.09,51.39
BANKS,7.64,-8.69,53.04
MISC,5.90,-18.85,74.21
PAINTS,5.67,-11.95,34.01
ELECTRICAL,5.44,1.48,45.23
HEALTHCARE,4.70,0.01,29.19
AUTO,4.28,-14.87,65.39


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2617406.0,26
XR,1087832.0,13
AR,1026543.0,10
X40,687133.0,10
X40N,527003.0,11
XY25,464886.0,8
OX40N,304059.0,11
SR,176634.0,2
MH,84320.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1958061.0,20
M-SC,1749618.0,21
H-LC,617800.0,12
X-LC,545786.0,8
H-MC,520685.0,8
X-MC,415178.0,6
M-MC,414695.0,3
X-SC,295239.0,5
M-LC,266391.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      881279.0      6
M-SC       XY24      794395.0      7
H-SC       AR        522869.0      3
M-SC       XR        336676.0      4
M-MC       XY24      299969.0      2
H-MC       XY24      295903.0      3
X-LC       X40       278072.0      3
H-SC       XR        215427.0      3
           X40N      200725.0      4
H-LC       X40N      197579.0      4
M-SC       AR        186423.0      2
           OX40N     162819.0      5
H-LC       X40       157916.0      3
X-MC       X40       157477.0      2
M-LC       XY24      156883.0      2
L-SC       XR        149781.0      2
X-SC       XR        120992.0      1
X-LC       AR        118829.0      1
M-SC       XY25      118298.0      1
M-MC       XR        114726.0      1
X-MC       XY25      104314.0      1
X-LC       XY25       99344.0      3
M-LC       XR         95630.0      1
X-SC       SR         94180.0      1
H-SC       OX40N      85556.0      3
X-MC       XY24       83003.0      1
M-SC       SR         82454.0      1
H-MC       XY25       81970.0      1
H-LC       AR         79693.0      1
           X5K        74738.0      2
           X200       69307.0      1
M-SC       X40        68553.0      1
H-MC       AR         60087.0      1
L-SC       AR         58642.0      1
L-MC       XR         54600.0      1
X-MC       X40N       53663.0      1
H-SC       MH         52205.0      1
X-LC       X40N       49541.0      1
L-LC       XY25       47082.0      1
X-SC       XY24       41104.0      1
           OX40N      38963.0      2
H-LC       XY24       38567.0      1
H-MC       MH         32115.0      1
           X40N       25495.0      1
           X40        25115.0      1
L-SC       XY24       23167.0      1
X-MC       OX40N      16721.0      1
M-LC       XY25       13878.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
